In [ ]:
import torch.utils.data
from utils.data import get_data_split
from utils.models import create_video_ResNet
from utils.trainner import train_silence
from utils.model_performance import plot_losses, get_predictions, calculate_metrics

# Config data

In [ ]:
BASE_PATH = "../../../datasets/ucf-101-Sampled-10-clases"
EPOCHS = 100
BATCH_SIZE = 4
TRANSFER_LEARNING_RATE = 3e-3
DEVICE = torch.device("cuda" if torch.cuda else "cpu")
DEVICE = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# Load data

In [ ]:
train_dataset, val_dataset, test_dataset, label_dict = get_data_split(base_path=BASE_PATH, batch_size=BATCH_SIZE, seq_lenght=60, num_workers=4)

In [ ]:
label_dict

# Transfer Learning

In [ ]:
model, preprocess = create_video_ResNet(n_classes=len(label_dict), device=DEVICE)

model.requires_grad = False
model.fc.requires_grad = True

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=TRANSFER_LEARNING_RATE)

train_loss, val_loss = train_silence(model=model, train_dataset=train_dataset, val_dataset=val_dataset, preprocess=preprocess, device=DEVICE, criterion=criterion, optimizer=optimizer, epochs=EPOCHS, num_classes=len(label_dict))

# load the best weights.
y_pred, y = get_predictions(model=model, dataloader=test_dataset, preprocess=preprocess, device=DEVICE)

plot_losses(train_loss, val_loss)
calculate_metrics(y, y_pred, labels=[v for k,v in label_dict.items()])